### Keras CNN 实现 CHF 判别

#### 1. 使用 conda 安装 Keras

` conda install keras `

#### 2. 导入 CNF 数据集

In [1]:
from PIL import Image
import os
import numpy

output_dir = 'output'

real_images = []
real_label = []

output_file = [name for name in os.listdir(output_dir) if name != '.DS_Store'][0]

for file_name in [name for name in os.listdir(output_dir+os.sep+output_file) if name != '.DS_Store']:
    real_images.append(list(Image.open(output_dir+os.sep+output_file+os.sep+file_name).getdata()))
    real_label.append(1)

x = numpy.array(real_images)
y = numpy.array(real_label)

#### 3. 处理数据集

* 测试模式下，有 40 个字，每种字套用 13 种字体

* 除以 255 是为了将 0-255 的 LA 值映射到 0-1 之间

In [2]:
x = x.reshape(x.shape[0],128,128,2).astype('float32') / 255

In [3]:
from keras.utils import np_utils
y = np_utils.to_categorical(y)

Using TensorFlow backend.


#### 4. 建立模型

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten

In [5]:
model = Sequential([
    Conv2D(input_shape=(128,128,2),
           filters=16,
           kernel_size=3,
           padding='same',
           activation='relu'),
    Dropout(0.25),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=32,
           kernel_size=5,
           padding='same',
           activation='relu'),
    Dropout(0.25),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=64,
           kernel_size=3,
           padding='same',
           activation='relu'),
    Dropout(0.25),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(units=1024,
          kernel_initializer='normal',
          activation='relu'),
    Dropout(0.5),
    Dense(units=128,
          kernel_initializer='normal',
          activation='relu'),
    Dense(units=2,
          kernel_initializer='normal',
          activation='softmax')
])

#### 5. 输出模型结构

In [6]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 16)      304       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        12832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        18496     
__________

#### 6. 定义训练方式

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#### 7. 开始训练

In [8]:
history = model.fit(x=x,
                   y=y,
                   validation_split=0.2,
                   epochs=10,
                   batch_size=32,
                   verbose=2)

Train on 10 samples, validate on 3 samples
Epoch 1/10
 - 4s - loss: 0.8836 - acc: 0.0000e+00 - val_loss: 0.2760 - val_acc: 1.0000
Epoch 2/10
 - 2s - loss: 0.0951 - acc: 1.0000 - val_loss: 0.0657 - val_acc: 1.0000
Epoch 3/10
 - 2s - loss: 0.0032 - acc: 1.0000 - val_loss: 0.0114 - val_acc: 1.0000
Epoch 4/10
 - 2s - loss: 3.8363e-05 - acc: 1.0000 - val_loss: 0.0016 - val_acc: 1.0000
Epoch 5/10
 - 2s - loss: 1.9670e-06 - acc: 1.0000 - val_loss: 2.0403e-04 - val_acc: 1.0000
Epoch 6/10
 - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 2.3862e-05 - val_acc: 1.0000
Epoch 7/10
 - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 2.6822e-06 - val_acc: 1.0000
Epoch 8/10
 - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 3.3776e-07 - val_acc: 1.0000
Epoch 9/10
 - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 10/10
 - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
